In [176]:
import warnings
warnings.filterwarnings(action='ignore')

import os
import pandas as pd
import numpy as np

from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
import IPython.display as ipd
import time

In [177]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import torch
import torchvision
import torch.nn.functional as F
from torchvision.transforms import functional
from torch import Tensor, nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import make_grid
import torchvision.transforms as transforms
import albumentations as A
from albumentations.pytorch import ToTensor

from efficientnet_pytorch import EfficientNet

In [221]:
import torch
import torchvision
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from adamp import AdamP
from torchsummary import summary

## 시드 고정

In [405]:
import random
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  
    torch.backends.cudnn.deterministic = True  
    torch.backends.cudnn.benchmark = True  
seed_everything()

## 데이터 불러오기

In [179]:
train = pd.read_csv('./train/train_data.csv')
test = pd.read_csv('./test/test/test_data.csv')

In [180]:
train.columns = ['file_name','label']

In [181]:
test.head()

,file_name
0,idx0001.png
1,idx0002.png
2,idx0003.png
3,idx0004.png
4,idx0005.png


In [355]:
class CustomDataset(torch.utils.data.Dataset): 
    def __init__(self,df,path,option,augmentation=None):
        self.df = df
        self.option = option
        self.augmentation = augmentation
        self.path = path

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx): 
        file_path  = self.df.iloc[idx,0]
        image=  Image.open(os.path.join(self.path,file_path)).convert('RGB')
        image = image.resize((100,100))
        image = np.array(image)
        
        #image = transforms.ToTensor()(image=image)['image']
        
        
        if self.augmentation is not None:
            image = self.augmentation(image=image)['image']
        
        
        if self.option =='train':
            label = self.df.iloc[idx,1]
            label = torch.tensor(label, dtype=torch.int64)
            return image, label
        
        return image
    

In [412]:
class MnistModel(nn.Module):
    def __init__(self,):
        super().__init__()
        #self.conv2d = nn.Conv2d(3, 3, 3, stride=1)
        self.model = torchvision.models.resnet34(pretrained = False)
        #self.model = EfficientNet.from_name('efficientnet-b1')
        self.dropout = nn.Dropout(0.5)
        
        self.classifier = nn.Linear(1000, 10)

    def forward(self, images):
        #outputs = self.conv2d(images)
        outputs = self.model(images)
        outputs = self.dropout(outputs)
        outputs = self.classifier(outputs)
        return outputs

In [413]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits = 5)
folds=[]
for trn_idx,val_idx in skf.split(train['file_name'],train['label']):
    folds.append((trn_idx,val_idx))

In [ ]:
num_epochs = 30

for fold in range(5):
    model = MnistModel().to('cuda')
    optimizer = AdamP(model.parameters(), lr=1e-3, )
    criterion = nn.CrossEntropyLoss()
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                step_size = 5,
                                                gamma = 0.9)
        
    best_models = [] # 폴드별로 가장 validation acc가 높은 모델 저장
    train_idx = folds[fold][0]
    valid_idx = folds[fold][1]
    
    trn_data = train.loc[trn_idx]
    val_data = train.loc[valid_idx]
    
    train_transforms = A.Compose([
    A.Normalize(),
    A.HorizontalFlip(),
    A.RandomRotate90(p=0.5),
    A.VerticalFlip(p=0.5),
    ToTensor()
    ])

    test_transforms = A.Compose([
        A.Normalize(),
        ToTensor()
        ])

    train_dataset = CustomDataset(train_data,'train/','train',train_transforms)
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=0)

    valid_dataset = CustomDataset(val_data,'train/','train',test_transforms)
    valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False, num_workers=0)
    
    
    valid_acc_max = 0.85
    for epoch in range(num_epochs):

        for i, (images, targets) in enumerate(train_loader):
            model.train()
            images, targets = images.to('cuda'), targets.to('cuda')

            optimizer.zero_grad()
            outputs = model(images)

            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()  

            if (i+1) % 20 == 0:
                print(f'Epoch: {epoch} - Loss: {loss:.6f}')


        val_loss = []
        val_acc = []
        for i, (images, targets) in enumerate(valid_loader):
            model.eval()
            images, targets = images.to('cuda'), targets.to('cuda')

            with torch.no_grad():
                outputs = model(images)
                valid_loss = criterion(outputs,targets).cpu().detach().numpy()

                preds = torch.argmax(outputs,axis = 1)
                preds = preds.cpu().detach().numpy()

                targets = targets.cpu().detach().numpy()
                batch_acc = (preds==targets).mean()



                val_loss.append(valid_loss)
                val_acc.append(batch_acc)

        val_loss = np.mean(val_loss)
        val_acc = np.mean(val_acc)

        print(f'Epoch: {epoch} - valid Loss: {val_loss:.6f} - valid_acc : {val_acc:.6f}')
    
    # Learning rate 조절
    lr_scheduler.step()
    
    if valid_acc_max < val_acc:
        valid_acc_max = val_acc
        best_models.append(model)
        
        

Epoch: 0 - Loss: 1.464996
Epoch: 0 - Loss: 1.204819
Epoch: 0 - Loss: 1.341240
Epoch: 0 - valid Loss: 1.552484 - valid_acc : 0.529883
Epoch: 1 - Loss: 0.900114
Epoch: 1 - Loss: 1.075518
Epoch: 1 - Loss: 0.789165
Epoch: 1 - valid Loss: 0.718386 - valid_acc : 0.741406
Epoch: 2 - Loss: 0.983739
Epoch: 2 - Loss: 0.863544
Epoch: 2 - Loss: 0.663585
Epoch: 2 - valid Loss: 0.770577 - valid_acc : 0.716992
Epoch: 3 - Loss: 0.569128
Epoch: 3 - Loss: 0.496609
Epoch: 3 - Loss: 0.899869
Epoch: 3 - valid Loss: 0.530221 - valid_acc : 0.801953
Epoch: 4 - Loss: 0.523583
Epoch: 4 - Loss: 0.474765


In [ ]:
test_transforms = A.Compose([
    A.Normalize(),
    ToTensor()
])

test_dataset = CustomDataset(test,'test/test','test',test_transforms)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=0)

In [305]:
model.eval()
y_pred = []
with torch.no_grad():
    for i, d in enumerate(test_loader):
        inputs = d.to('cuda')
        outputs = model(inputs).detach().cpu().numpy()
        y_pred.extend(outputs.argmax(axis=1).astype(int))
        
        
        
#y_pred = np.concatenate(y_pred, 0)

In [306]:
y_pred

[7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,


In [217]:
submission = pd.read_csv('sample_submission.csv')
submission['label'] = y_pred

In [218]:
submission.to_csv('sub_211122.csv',index = False)

In [219]:
submission['label'].value_counts()

3    593
7    557
1    557
9    527
6    521
5    483
0    469
4    458
8    429
2    406
Name: label, dtype: int64